<a href="https://colab.research.google.com/github/juansuribemo/Regresion_lineal/blob/main/Regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 3: Regresión Lineal y Conceptos Fundamentales




## Instrucciones Generales

**Fecha de entrega:** 20 de septiembre.
**Tiempo disponible:** Tienen dos clases y el fin de semana para completar el taller.

1.  **Trabajo en Grupo:** Este taller se debe realizar en los grupos previamente definidos.
2.  **Repositorio en GitHub:** Cada grupo debe crear un repositorio en GitHub.
    * Una sola persona del grupo es responsable de crearlo.
    * El repositorio debe llamarse **`Regresion_Lineal`**.
    * Este archivo debe ser subido al repositorio con el nombre **`Regresion_Lineal.ipynb`**.
    * **Alternativa:** Si tienen complicaciones con GitHub, pueden enviar el enlace del cuaderno de Google Colab al aula virtual.
3.  **Actividad a Mano (20% de la nota):**
    * La asistencia a clase es **obligatoria**.
    * Se realizará una actividad práctica de regresión lineal a mano.
    * Cada integrante del grupo deberá firmar la hoja de la actividad, la cual constituye el 20% de la nota de este taller.

---

## **Parte 1: Definición del Problema**

Piensa en tu experiencia académica o laboral en un área de operaciones y elige **un problema** relacionado con inventarios, calidad, mantenimiento o productividad.

1.  Indica cuál es tu variable dependiente **$Y$** (aquello que quieres predecir o clasificar).
2.  Lista entre 3 y 5 variables independientes **$X$** que medirías para predecir $Y$.
3.  Especifica si tu problema es de **regresión** (predecir un valor numérico) o de **clasificación**.

Problema elegido: Optimización del nivel de inventario de rollos de tela y productos semi-terminados en la planta (evitar roturas de línea por falta de material y reducir stock inmovilizado).


**Respuesta 1:** Variable dependiente (Y): Nivel de inventario óptimo por SKU (rollos de tela o semi-terminado), en metros (o unidades de rollo) por período (ej. por semana).


**Respuesta 2:** Demanda real de producción (X₁) — metros requeridos por semana por planificación de órdenes (unidades: metros/semana). Proviene del MRP / órdenes productivas.
Lead time proveedor digitalizado (X₂) — tiempo medio de reposición medido en días (incluye trazabilidad por sensores de transporte). (días)
Tasa de fallas en línea / scrap (X₃) — metros perdidos por defectos o rechazos en producción por semana (metros/semana).

Capacidad efectiva de la línea (X₄) — metros/hora útiles (considera paradas, setups detectadas por IIoT). (m/h)

Variabilidad de la demanda (X₅) — desviación estándar semanal de la demanda en los últimos N semanas (metros).

**Respuesta 3:** Tipo de problema: Regresión — predecir un valor numérico (nivel de inventario por SKU).


---

## **Parte 2: Preprocesamiento de Datos y *Leakage***

Basado en el caso que definiste en la Parte 1:

1.  Lista entre 3 y 5 **transformaciones** que aplicarías a tus datos (ej. imputación de valores faltantes, codificación de variables categóricas, escalado, creación de *lags*, etc.) y **justifica por qué** cada una es necesaria.
2.  Señala un posible riesgo de ***data leakage*** (fuga de datos) en tu plan y explica cómo lo evitarías usando un *pipeline* de preprocesamiento.

**Respuesta :** :1.Imputación de valores faltantes por sensor:
 rellenar faltantes en series temporales de consumo o lecturas de IIoT (p. ej. media móvil o interpolación por tiempo).
 Por qué: los sensores pueden fallar intermitentemente; perder ventanas de tiempo puede sesgar la estimación de consumo/lead time.
 2.Creación de lags y features temporales:
 Incluir demanda_t-1, demanda_t-2, promedio móvil 4 semanas, y estacionalidad semanal/mensual.
 Por qué: la demanda textil suele mostrar estacionalidad (colecciones, campañas) y la información histórica inmediata es predictiva.
 Codificación de variables categóricas de proveedor/fabricante:
 one-hot para proveedor, tipo de tejido (tejido plano/tejido de punto), línea de producción.
 Por qué: distintos proveedores/tejidos tienen lead times y tasas de scrap diferentes.
 3.Escalado (standardization) de variables continuas:
Estandarizar (media 0, varianza 1) para modelos que usan regularización o distancias.
 Por qué: lead times en días vs metros/hora vs costos están en escalas muy distintas; mejora convergencia y coeficientes interpretables.
4.Tratamiento de outliers por eventos (p. ej. parada total por incendio/lockdown):
Detectar y marcar eventos (flag) o winsorizar.
 Por qué: eventos extremos no representativos distorsionan el modelo; pero conviene conservar la información etiquetada como evento.
# 2. Riesgo de data leakage y cómo evitarlo con pipeline:
Riesgo: usar información futura inadvertida — p. ej. calcular la media móvil usando datos hasta el futuro al crear una variable para el período t (o usar demanda real posterior para ajustar inventario previsto). Otro riesgo: incluir en features indicadores derivados de decisiones tomadas después de conocer la demanda (como órdenes de emergencia colocadas tras una rotura).
Prevención (pipeline):
Separación temporal train/test/validation por fecha (no muestreo aleatorio): el test debe representar períodos posteriores.
Pipeline encadenado que ajusta tranformaciones sólo en train: todas las transformaciones (imputación, scaling, encoding, creación de lags) se fit-ean exclusivamente en datos de entrenamiento y luego se transforman el validation/test sin recalcular parámetros usando esos splits.
 TimeSeriesSplit o backtesting: usar validación temporal (folds deslizantes) para evaluar robustez sin leakage.
Auditoría de features: marcar explícitamente features que usan información de demanda futura y prohibir su inclusión.


---

## **Parte 3: Interpretación y Métricas de Regresión Simple**

Para esta sección, elige un caso simple de regresión (puede ser el tuyo o uno hipotético, como predecir la demanda de un producto según su precio).

1.  Define claramente las variables **$Y$** y **$X$** junto con sus **unidades** (ej. $Y$: número de unidades vendidas, $X$: precio en dólares).
2.  Supón que entrenas un modelo y obtienes una pendiente de **$\hat{\beta}_1 = -0.6$**. Escribe una interpretación clara y concisa de este coeficiente en el contexto de tu problema.
3.  ¿Qué **métrica** de evaluación usarías (MAE, RMSE, o MAPE) y **por qué** es la más adecuada para tu caso?
4.  Menciona **un supuesto** del modelo de regresión lineal que validarías (ej. linealidad, homocedasticidad) y explica **cómo** lo harías (usando un gráfico o una prueba estadística).

**Respuesta :**Caso simple elegido: Predecir demanda semanal de un SKU de tela según su precio de venta al producto interno (cuando la planta calcula consumo interno por precio de venta interno/cliente).

Y: Demanda semanal del SKU (metros/semana).

X: Precio promedio aplicado en la semana (peso colombiano por metro, COP/m).

Supongamos que la pendiente estimada β₁ = −120.

Interpretación clara: Manteniendo todo lo demás constante, por cada aumento de 1 COP en el precio por metro, la demanda esperada esa semana disminuye en 120 metros. (Un efecto negativo: precio ↑ → demanda ↓).

Ejemplo práctico: si el precio sube 10 COP, esperaríamos 1,200 metros menos por semana.

Métrica de evaluación elegida: MAPE (Mean Absolute Percentage Error).

Por qué: en control de inventarios y demanda nos interesa un error relativo (%), porque los SKUs varían mucho en magnitud (algunos SKU tienen 10 m/semana, otros 10,000 m/semana). MAPE facilita comparar errores entre SKU y priorizar ajustes (por ejemplo, SKU con MAPE alto exigen modelos separados).

Nota: si hay muchos valores cercanos a cero, MAPE puede ser inestable; en esos casos preferiría MAE o SMAPE.

Supuesto del modelo lineal a validar: Linealidad entre X y Y.

Cómo validarlo: graficar residuos vs valores predichos: si no hay patrón (los residuos se distribuyen alrededor de 0 sin tendencia), la linealidad es aceptable. También podría mirar un scatter X vs Y con la línea de regresión. Si sospecho no linealidad, probar transformaciones (log, polinomio) o modelos no lineales.


---

## **Parte 4: Regresión Múltiple y Colinealidad**

Volviendo a tu caso de la Parte 1 (con múltiples variables).

1.  Escribe el **vector de variables** $\vec{X}$ y la **respuesta** $Y$.
2.  Explica cómo interpretarías el **coeficiente** de una de tus variables clave (incluyendo unidades y el sentido de la relación: positiva o negativa).
3.  Si sospecharas que existe **colinealidad** entre tus variables, menciona **dos acciones** que podrías tomar para mitigarla.

**Respuesta:**# Vector de variables (X) y respuesta (Y):
  𝑋=[𝑋1
     𝑋2
     𝑋3
     𝑋4
     𝑋5]
 donde
# 𝑋1= Demanda real de producción (m/semana)
# X2= Lead time proveedor (días)
# X3= Tasa de scrap (m/semana)
# X4= Capacidad efectiva de línea (m/h)
# X5= Variabilidad de la demanda (σ, m)
# Y = Nivel de inventario óptimo (metros por periodo).
# Interpretación de un coeficiente clave (ejemplo: coef. de X₂, lead time):
# Supongamos coeficiente
# 𝛽2=15 (unidad: metros/día).
# Interpretación: manteniendo las otras variables constantes, cada día adicional de lead time del proveedor requiere en promedio 15 metros adicionales de inventario para mantener el mismo nivel de servicio. Relación positiva: mayor retraso → mayor inventario de seguridad.
# -Si sospecho colinealidad entre variables, dos acciones para mitigarla:
# Calcular VIF y eliminar o consolidar variables con VIF alto: por ejemplo, si Demanda histórica y Capacidad efectiva están muy correlacionadas (ambas reflejan carga), podría mantener sólo la más predictiva o combinar en una métrica compuesta (ratio de demanda/capacidad).
# -Usar regularización (Ridge o Lasso): Ridge reduce varianza de coeficientes colineales; Lasso puede hacer selección de variables eliminando algunas.


---

## **Parte 5: Interacciones y Multicolinealidad (VIF)**

1.  Plantea un caso con una variable $Y$ y entre 4 y 6 variables $X$. ¿Qué término de **interacción** entre dos variables podrías añadir al modelo y **por qué** crees que sería útil?
2.  Si al calcular el Factor de Inflación de la Varianza (VIF) para una variable, obtienes un valor alto (ej. > 10), menciona **dos acciones** que podrías tomar para solucionarlo.

**Respuesta :**
Y = Nivel inventario (m)

 𝑋={Demanda historica,Lead time,Tasa scrap,Velocidad lınea,Tipo tejido (codificado),Stock en transito}

Término de interacción sugerido:

Demanda histórica × Lead time (X_demand × X_leadtime).

Por qué: si la demanda histórica es alta y además el lead time es largo, el efecto en el inventario requerido es más que la suma individual (es multiplicativo): alta demanda con lead times largos multiplica la necesidad de inventario de seguridad. Incluir este término permite captar ese efecto conjunto.

1.Si al calcular VIF una variable tiene VIF > 10, dos acciones a tomar:
Centrare (mean-center) las variables continuas antes de crear interacciones: centrar reduce multicolinealidad entre términos principales e interactivos (por ejemplo, usar 𝑋−𝑋).

Eliminar o combinar variables correlacionadas / aplicar PCA: crear componentes principales para representar la sub-estructura (p. ej. combinar medidas de capacidad en un índice) o eliminar variables menos relevantes tras análisis. Otra opción es usar Ridge si no quiero eliminar la variable.

---

## **Parte 6: Variables Categóricas e Interacciones**

1.  Define una **variable categórica** para tu caso (puedes inventarla si no la tenías). Elige una de sus categorías como el nivel **base** o de referencia y **justifica** tu elección.
2.  Crea una **interacción** entre una variable numérica y la variable categórica que definiste. Explica cómo se interpretaría el coeficiente de esta interacción.

**Respuesta:**
Variable categórica: Tipo de tejido = {Woven (tejido plano), Knit (tejido de punto), Nonwoven (no tejido)}.

Nivel base (referencia): Woven (tejido plano).

Justificación: es un tipo de tejido mayoritario en la línea y tiene comportamiento de producción más estable (menor variabilidad y scrap) — usarlo como referencia facilita comparar cómo difieren Knit y Nonwoven.

Interacción propuesta: Velocidad_línea × Tipo_tecido_Knit (donde Tipo_tecido_Knit = 1 si Knit, 0 si referencia Woven).

 Interpretación del coeficiente de la interacción (β_inter):

Supongamos β_inter = −2 (unidades: metros de inventario por (m/h) adicional cuando es Knit).

Qué significa: el efecto de aumentar la velocidad de la línea en la necesidad de inventario es diferente para Knit que para Woven. Con β_inter negativo, un aumento de 1 m/h en la velocidad reduce el inventario requerido en 2 metros adicionales sólo para Knit respecto al efecto que tendría para Woven. Es decir, el impacto de la velocidad depende del tipo de tejido.





---

## **Parte 7: Conceptos Clave de Clasificación**

Aunque el taller se centra en regresión, estos conceptos son fundamentales en Machine Learning.

1.  Explica qué es la **curva ROC** y para qué se utiliza en un problema de clasificación.
2.  Define el concepto de **accuracy** (exactitud) y menciona una situación en la que podría ser una métrica engañosa.
3.  Describe qué es una **matriz de confusión** y cómo se interpretan sus componentes (Verdaderos Positivos, Falsos Positivos, Verdaderos Negativos, Falsos Negativos).

**Respuesta:**
Curva ROC (Receiver Operating Characteristic):

Qué es: gráfico que pone en el eje vertical la tasa de verdaderos positivos (TPR = sensibilidad) y en el eje horizontal la tasa de falsos positivos (FPR = 1 − especificidad) para todos los umbrales posibles del clasificador (p. ej. probabilidad de defecto > t).

Para qué se usa: evaluar la capacidad discriminativa del detector de defectos (si sabe distinguir bien entre rollos defectuosos y buenos) independientemente del umbral. El AUC-ROC resume el rendimiento: 1 = perfecto, 0.5 = azar.

Accuracy (exactitud):

Definición: proporción de predicciones correctas = (TP + TN) / (TP+TN+FP+FN).

Cuándo es engañosa: si las clases están desbalanceadas — p. ej. sólo 1% de rollos tienen defectos, un modelo que predice “no defecto” siempre tendría 99% de accuracy pero sería inútil para detectar defectos (alto FN). En esos casos se prefieren recall, precision, F1 o curvas ROC/PR.

Matriz de confusión:

Tabla 2×2 con predicción vs realidad. Componentes:

TP (Verdaderos Positivos): rollo con defecto y modelo lo clasificó como defecto (correcto).

FP (Falsos Positivos): rollo bueno que el modelo clasificó como defectuoso (falso alarma).

TN (Verdaderos Negativos): rollo bueno y modelo dijo “bueno”.

FN (Falsos Negativos): rollo defectuoso que el modelo clasificó como bueno (fallo crítico).

Interpretación práctica: para una planta textil, FN puede ser más costoso (cliente recibe defectos) que FP (inspección manual extra), por lo tanto a menudo optimizamos para minimizar FN (maximizar recall) manteniendo precision razonable.


